In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer
from sklearn.linear_model import Lasso

In [2]:
# Define parameters for LASSO
alpha = [0.0001, 0.001, 0.01, 0.1, 1, 10]
#max_iter = [500, 1000, 10000, 100000]
selection = ['cyclic', 'random']
fit_intercept = [True, False]
tol = [0.0001, 0.001, 0.01, 0.1, 1, 10]
warm_start = [True, False]
#positive = [True, False]
#random_state = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
copy_X = [True, False]

# Create the random grid
random_grid = {'alpha': alpha,
               #'max_iter': max_iter,
               'selection': selection,
               'fit_intercept': fit_intercept,
               'tol': tol,
               'warm_start': warm_start,
               #'positive': positive,
               #'random_state': random_state,
               'copy_X': copy_X}

In [3]:
#Define MASE Metric
def mean_absolute_scaled_error(y_true, y_pred):
    mase=0
    # Define numerator as the forecast error
    numerator = (np.abs(y_true - y_pred))

    # Define denominator as the mean absolute error of the in-sample one-step naive forecast
    y_true_ohne_1 = y_true[1:].reset_index(drop=True)
    y_true_ohne_ende = y_true[:-1].reset_index(drop=True)
    denominator = np.mean(np.abs(y_true_ohne_1 - y_true_ohne_ende))

    mase = np.mean(np.abs(numerator / denominator))

    return mase

scorer_mase= make_scorer(mean_absolute_scaled_error, greater_is_better=False)


In [4]:
# Load the data
data = pd.read_parquet('/Users/paddy/Documents/GitHub/Masterthesis_ML/data/03_15min_dataset.parquet')

# Convert the date column to datetime
data['date'] = pd.to_datetime(data['date']) #,format='%d/%m/%y %H:%M:%S').dt.strftime('%Y-%m-%d %H:%M:%S') 


In [5]:
#Feature engineering
# Create a new column for the time
data['time'] = [x for x in range(0, len(data))]

# Drop nan values
data = data.dropna()

# Rename column count to y
data = data.rename(columns={'count': 'y'})


In [6]:
# Set y to the last column
cols = list(data.columns.values) #Make a list of all of the columns in the df
cols.pop(cols.index('y')) #Remove y from list
data = data[cols+['y']] #Create new dataframe with columns in the order you want

# Define the feature columns and the target column
feature_cols = ['time']
target_col = 'y'

# drop the date column
train_data = np.delete(data, 0, 1) 

# Split the data into X and y
X_train, y_train = train_data[:, :-1], train_data[:, -1]


In [7]:
#vgl: https://lightrun.com/answers/scikit-learn-scikit-learn-grid_search-feeding-parameters-to-scorer-functions

# X and y to pandas dataframe
X_train = pd.DataFrame(X_train)
y_train = pd.Series(y_train)


In [8]:
# Set Cross Validation to 5 iterations
tscv = TimeSeriesSplit(n_splits=5)

In [9]:
# Instantiate model
model = Lasso()

search = RandomizedSearchCV(estimator = model, 
                               param_distributions = random_grid, 
                               n_iter = 25, 
                               cv = tscv,
                               refit=True, 
                               verbose=3, 
                               random_state=42, 
                               n_jobs = -1, 
                               scoring= scorer_mase, #make_scorer(scorer_mase, greater_is_better=True), #'neg_root_mean_squared_error', #
                               error_score=np.nan)


In [10]:
# Fit the random search model
search.fit(X_train, y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits


[CV 1/5] END alpha=0.01, copy_X=True, fit_intercept=False, selection=random, tol=0.1, warm_start=True;, score=-2.504 total time=   0.0s
[CV 5/5] END alpha=0.01, copy_X=True, fit_intercept=False, selection=random, tol=0.1, warm_start=True;, score=-1.670 total time=   0.0s
[CV 4/5] END alpha=0.001, copy_X=True, fit_intercept=True, selection=random, tol=10, warm_start=True;, score=-1.439 total time=   0.0s
[CV 4/5] END alpha=0.01, copy_X=True, fit_intercept=False, selection=random, tol=0.1, warm_start=True;, score=-1.848 total time=   0.0s
[CV 3/5] END alpha=0.01, copy_X=True, fit_intercept=False, selection=random, tol=0.1, warm_start=True;, score=-1.978 total time=   0.0s
[CV 5/5] END alpha=0.001, copy_X=True, fit_intercept=True, selection=random, tol=10, warm_start=True;, score=-1.396 total time=   0.0s
[CV 2/5] END alpha=0.01, copy_X=True, fit_intercept=False, selection=random, tol=0.1, warm_start=True;, score=-2.209 total time=   0.0s
[CV 1/5] END alpha=0.1, copy_X=False, fit_intercep

RandomizedSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None),
                   estimator=Lasso(), n_iter=25, n_jobs=-1,
                   param_distributions={'alpha': [0.0001, 0.001, 0.01, 0.1, 1,
                                                  10],
                                        'copy_X': [True, False],
                                        'fit_intercept': [True, False],
                                        'selection': ['cyclic', 'random'],
                                        'tol': [0.0001, 0.001, 0.01, 0.1, 1,
                                                10],
                                        'warm_start': [True, False]},
                   random_state=42,
                   scoring=make_scorer(mean_absolute_scaled_error, greater_is_better=False),
                   verbose=3)

In [11]:
print('Best Score: %s' % search.best_score_)
print('Best Hyperparameters: %s' % search.best_params_)
print('Best Model: %s' % search.best_estimator_)
print('Best Index: %s' % search.best_index_)
print('CV Results: %s' % search.cv_results_)
print('Refit Time: %s' % search.refit_time_)
print('Scorer: %s' % search.scorer_)

Best Score: -1.4322685083366204
Best Hyperparameters: {'warm_start': True, 'tol': 0.1, 'selection': 'random', 'fit_intercept': True, 'copy_X': True, 'alpha': 10}
Best Model: Lasso(alpha=10, selection='random', tol=0.1, warm_start=True)
Best Index: 3
CV Results: {'mean_fit_time': array([0.00435357, 0.00512695, 0.00466137, 0.00310712, 0.00280323,
       0.00281091, 0.00259657, 0.00278707, 0.00274525, 0.00261755,
       0.00330729, 0.00264044, 0.00255885, 0.00261073, 0.00273132,
       0.00265813, 0.00261979, 0.00244961, 0.00242   , 0.00296941,
       0.00251842, 0.00304623, 0.00298681, 0.00258322, 0.00263915]), 'std_fit_time': array([0.00176335, 0.00240452, 0.0014381 , 0.00158978, 0.00090356,
       0.00055657, 0.00064631, 0.00103921, 0.00072697, 0.00061689,
       0.00151945, 0.00083379, 0.00068331, 0.00081548, 0.00062124,
       0.00065368, 0.00080827, 0.00070607, 0.00069587, 0.00116249,
       0.00075474, 0.00089335, 0.00098227, 0.00065784, 0.00065099]), 'mean_score_time': array([0.00